In this notebook we would be trying to make a car(agent) learn to solve(self-drive) around a race track(environment) using **Reinforcement Learning**.

## Import Dependencies

In [2]:
# !pip install gym[box2d] pyglet==1.5.26
# follow this: https://www.youtube.com/watch?v=e3DyCg0fgx0
# !pip install stable-baselines3[extra]

In [3]:
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
import os

## Test Environment

In [5]:
# https://www.gymlibrary.ml/environments/box2d/car_racing/
# https://github.com/openai/gym/blob/master/gym/envs/box2d/car_racing.py
environment_name = 'CarRacing-v2'
env = gym.make(environment_name, render_mode="human")

In [9]:
env.action_space

Box([-1.  0.  0.], 1.0, (3,), float32)

In [10]:
env.observation_space

Box(0, 255, (96, 96, 3), uint8)

In [11]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print(f'Episode:{episode} Score:{score}')
env.close()

Episode:1 Score:-37.08609271523239
Episode:2 Score:-34.02777777777824
Episode:3 Score:-23.954372623574336
Episode:4 Score:-23.07692307692328
Episode:5 Score:-30.313588850174643


## Train Model

In [6]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])

In [7]:
model = PPO("CnnPolicy", env, verbose=1)

Using cpu device
Wrapping the env in a VecTransposeImage.


In [8]:
model.learn(total_timesteps=5) # Just to show how a model is trained
# later we will train model over millions of timesteps

-----------------------------
| time/              |      |
|    fps             | 64   |
|    iterations      | 1    |
|    time_elapsed    | 31   |
|    total_timesteps | 2048 |
-----------------------------


## Save and Load model

In [22]:
PPO_path = os.path.join('Training', 'Saved Models', 'PPO_model')

In [11]:
model.save(PPO_path)

In [12]:
del model

In [13]:
model = PPO.load(PPO_path, env=env)

Wrapping the env in a VecTransposeImage.


## Evaluation

In [9]:
evaluate_policy(model, env, n_eval_episodes=10)

C:\Users\syeda\anaconda3\lib\site-packages\stable_baselines3\common\evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(4.68673897460103, 39.74052565126847)

## Test model

In [31]:
obs = env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    env.render()
    if done: 
        print('info', info)
        break
env.close()

info [{'TimeLimit.truncated': True, 'terminal_observation': array([[[100, 202, 100],
        [100, 202, 100],
        [100, 202, 100],
        ...,
        [100, 202, 100],
        [100, 202, 100],
        [100, 202, 100]],

       [[100, 202, 100],
        [100, 202, 100],
        [100, 202, 100],
        ...,
        [100, 202, 100],
        [100, 202, 100],
        [100, 202, 100]],

       [[100, 202, 100],
        [100, 202, 100],
        [100, 202, 100],
        ...,
        [100, 202, 100],
        [100, 202, 100],
        [100, 202, 100]],

       ...,

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0, 

## Train for longer timesteps

In [39]:
# 430k Time steps
PPO_path = os.path.join('Training', 'Saved Models', 'PPO_430K_Driving_model')

model.learn(total_timesteps=430000)

model.save(PPO_path)

-----------------------------
| time/              |      |
|    fps             | 28   |
|    iterations      | 1    |
|    time_elapsed    | 71   |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 22           |
|    iterations           | 2            |
|    time_elapsed         | 180          |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0062228646 |
|    clip_fraction        | 0.0748       |
|    clip_range           | 0.2          |
|    entropy_loss         | -4.23        |
|    explained_variance   | 0.00124      |
|    learning_rate        | 0.0003       |
|    loss                 | 0.176        |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.00741     |
|    std                  | 0.985        |
|    value_loss           | 0.594        |
----------------

-----------------------------------------
| time/                   |             |
|    fps                  | 30          |
|    iterations           | 12          |
|    time_elapsed         | 794         |
|    total_timesteps      | 24576       |
| train/                  |             |
|    approx_kl            | 0.020070236 |
|    clip_fraction        | 0.191       |
|    clip_range           | 0.2         |
|    entropy_loss         | -3.95       |
|    explained_variance   | 0.664       |
|    learning_rate        | 0.0003      |
|    loss                 | -0.0149     |
|    n_updates            | 110         |
|    policy_gradient_loss | -0.0181     |
|    std                  | 0.898       |
|    value_loss           | 0.125       |
-----------------------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 31         |
|    iterations           | 13         |
|    time_elapsed         | 848       

-----------------------------------------
| time/                   |             |
|    fps                  | 34          |
|    iterations           | 23          |
|    time_elapsed         | 1384        |
|    total_timesteps      | 47104       |
| train/                  |             |
|    approx_kl            | 0.026520796 |
|    clip_fraction        | 0.209       |
|    clip_range           | 0.2         |
|    entropy_loss         | -3.64       |
|    explained_variance   | 0.92        |
|    learning_rate        | 0.0003      |
|    loss                 | -0.00693    |
|    n_updates            | 220         |
|    policy_gradient_loss | -0.0219     |
|    std                  | 0.811       |
|    value_loss           | 0.19        |
-----------------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 34          |
|    iterations           | 24          |
|    time_elapsed         | 1438  

-----------------------------------------
| time/                   |             |
|    fps                  | 26          |
|    iterations           | 34          |
|    time_elapsed         | 2613        |
|    total_timesteps      | 69632       |
| train/                  |             |
|    approx_kl            | 0.036464922 |
|    clip_fraction        | 0.293       |
|    clip_range           | 0.2         |
|    entropy_loss         | -3.3        |
|    explained_variance   | 0.934       |
|    learning_rate        | 0.0003      |
|    loss                 | -0.0315     |
|    n_updates            | 330         |
|    policy_gradient_loss | -0.036      |
|    std                  | 0.721       |
|    value_loss           | 0.219       |
-----------------------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 26         |
|    iterations           | 35         |
|    time_elapsed         | 2667      

----------------------------------------
| time/                   |            |
|    fps                  | 28         |
|    iterations           | 45         |
|    time_elapsed         | 3210       |
|    total_timesteps      | 92160      |
| train/                  |            |
|    approx_kl            | 0.09077087 |
|    clip_fraction        | 0.459      |
|    clip_range           | 0.2        |
|    entropy_loss         | -2.95      |
|    explained_variance   | 0.872      |
|    learning_rate        | 0.0003     |
|    loss                 | 0.598      |
|    n_updates            | 440        |
|    policy_gradient_loss | -0.0175    |
|    std                  | 0.651      |
|    value_loss           | 3.04       |
----------------------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 28         |
|    iterations           | 46         |
|    time_elapsed         | 3267       |
|    total_times

...

----------------------------------------
| time/                   |            |
|    fps                  | 33         |
|    iterations           | 166        |
|    time_elapsed         | 10296      |
|    total_timesteps      | 339968     |
| train/                  |            |
|    approx_kl            | 0.13184397 |
|    clip_fraction        | 0.492      |
|    clip_range           | 0.2        |
|    entropy_loss         | -2.89      |
|    explained_variance   | 0.962      |
|    learning_rate        | 0.0003     |
|    loss                 | 0.0637     |
|    n_updates            | 1650       |
|    policy_gradient_loss | 0.0305     |
|    std                  | 0.644      |
|    value_loss           | 2.46       |
----------------------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 33         |
|    iterations           | 167        |
|    time_elapsed         | 10350      |
|    total_times

----------------------------------------
| time/                   |            |
|    fps                  | 33         |
|    iterations           | 177        |
|    time_elapsed         | 10928      |
|    total_timesteps      | 362496     |
| train/                  |            |
|    approx_kl            | 0.16847982 |
|    clip_fraction        | 0.545      |
|    clip_range           | 0.2        |
|    entropy_loss         | -2.72      |
|    explained_variance   | 0.82       |
|    learning_rate        | 0.0003     |
|    loss                 | 0.439      |
|    n_updates            | 1760       |
|    policy_gradient_loss | 0.0188     |
|    std                  | 0.614      |
|    value_loss           | 7.39       |
----------------------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 33         |
|    iterations           | 178        |
|    time_elapsed         | 10987      |
|    total_times

-----------------------------------------
| time/                   |             |
|    fps                  | 33          |
|    iterations           | 188         |
|    time_elapsed         | 11544       |
|    total_timesteps      | 385024      |
| train/                  |             |
|    approx_kl            | 0.071627535 |
|    clip_fraction        | 0.408       |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.59       |
|    explained_variance   | 0.954       |
|    learning_rate        | 0.0003      |
|    loss                 | 0.25        |
|    n_updates            | 1870        |
|    policy_gradient_loss | -0.00799    |
|    std                  | 0.589       |
|    value_loss           | 1.68        |
-----------------------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 33         |
|    iterations           | 189        |
|    time_elapsed         | 11598     

---------------------------------------
| time/                   |           |
|    fps                  | 33        |
|    iterations           | 199       |
|    time_elapsed         | 12188     |
|    total_timesteps      | 407552    |
| train/                  |           |
|    approx_kl            | 0.1769662 |
|    clip_fraction        | 0.566     |
|    clip_range           | 0.2       |
|    entropy_loss         | -2.53     |
|    explained_variance   | 0.877     |
|    learning_rate        | 0.0003    |
|    loss                 | 0.752     |
|    n_updates            | 1980      |
|    policy_gradient_loss | 0.0227    |
|    std                  | 0.581     |
|    value_loss           | 7.54      |
---------------------------------------
--------------------------------------
| time/                   |          |
|    fps                  | 33       |
|    iterations           | 200      |
|    time_elapsed         | 12247    |
|    total_timesteps      | 409600   |
| trai

----------------------------------------
| time/                   |            |
|    fps                  | 33         |
|    iterations           | 210        |
|    time_elapsed         | 12820      |
|    total_timesteps      | 430080     |
| train/                  |            |
|    approx_kl            | 0.16902441 |
|    clip_fraction        | 0.538      |
|    clip_range           | 0.2        |
|    entropy_loss         | -2.54      |
|    explained_variance   | 0.936      |
|    learning_rate        | 0.0003     |
|    loss                 | 0.255      |
|    n_updates            | 2090       |
|    policy_gradient_loss | 0.0137     |
|    std                  | 0.58       |
|    value_loss           | 5.66       |
----------------------------------------


In [40]:
model = PPO.load(PPO_path, env=env)

Wrapping the env in a VecTransposeImage.


In [41]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

(246.6938610434532, 141.88138320834923)

In [8]:
# 2M Time steps
PPO_path = os.path.join('Training', 'Saved Models', 'PPO_2M_Driving_model')

model.learn(total_timesteps=2000000)

model.save(PPO_path)

-----------------------------
| time/              |      |
|    fps             | 57   |
|    iterations      | 1    |
|    time_elapsed    | 35   |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 44           |
|    iterations           | 2            |
|    time_elapsed         | 91           |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0060943174 |
|    clip_fraction        | 0.0728       |
|    clip_range           | 0.2          |
|    entropy_loss         | -4.24        |
|    explained_variance   | 0.000735     |
|    learning_rate        | 0.0003       |
|    loss                 | 0.135        |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.00551     |
|    std                  | 0.992        |
|    value_loss           | 0.563        |
----------------

-----------------------------------------
| time/                   |             |
|    fps                  | 37          |
|    iterations           | 13          |
|    time_elapsed         | 708         |
|    total_timesteps      | 26624       |
| train/                  |             |
|    approx_kl            | 0.018434055 |
|    clip_fraction        | 0.178       |
|    clip_range           | 0.2         |
|    entropy_loss         | -3.97       |
|    explained_variance   | 0.873       |
|    learning_rate        | 0.0003      |
|    loss                 | 0.079       |
|    n_updates            | 120         |
|    policy_gradient_loss | -0.0173     |
|    std                  | 0.903       |
|    value_loss           | 0.19        |
-----------------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 37          |
|    iterations           | 14          |
|    time_elapsed         | 762   

-----------------------------------------
| time/                   |             |
|    fps                  | 37          |
|    iterations           | 24          |
|    time_elapsed         | 1308        |
|    total_timesteps      | 49152       |
| train/                  |             |
|    approx_kl            | 0.024409294 |
|    clip_fraction        | 0.225       |
|    clip_range           | 0.2         |
|    entropy_loss         | -3.62       |
|    explained_variance   | 0.901       |
|    learning_rate        | 0.0003      |
|    loss                 | 0.144       |
|    n_updates            | 230         |
|    policy_gradient_loss | -0.0296     |
|    std                  | 0.805       |
|    value_loss           | 0.489       |
-----------------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 37          |
|    iterations           | 25          |
|    time_elapsed         | 1363  

-----------------------------------------
| time/                   |             |
|    fps                  | 37          |
|    iterations           | 35          |
|    time_elapsed         | 1905        |
|    total_timesteps      | 71680       |
| train/                  |             |
|    approx_kl            | 0.033986017 |
|    clip_fraction        | 0.232       |
|    clip_range           | 0.2         |
|    entropy_loss         | -3.31       |
|    explained_variance   | 0.972       |
|    learning_rate        | 0.0003      |
|    loss                 | 0.137       |
|    n_updates            | 340         |
|    policy_gradient_loss | -0.0167     |
|    std                  | 0.731       |
|    value_loss           | 0.55        |
-----------------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 37          |
|    iterations           | 36          |
|    time_elapsed         | 1959  

----------------------------------------
| time/                   |            |
|    fps                  | 37         |
|    iterations           | 46         |
|    time_elapsed         | 2499       |
|    total_timesteps      | 94208      |
| train/                  |            |
|    approx_kl            | 0.04276219 |
|    clip_fraction        | 0.29       |
|    clip_range           | 0.2        |
|    entropy_loss         | -3.12      |
|    explained_variance   | 0.971      |
|    learning_rate        | 0.0003     |
|    loss                 | 0.539      |
|    n_updates            | 450        |
|    policy_gradient_loss | -0.00822   |
|    std                  | 0.688      |
|    value_loss           | 2.59       |
----------------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 37          |
|    iterations           | 47          |
|    time_elapsed         | 2553        |
|    total_

...

-----------------------------------------
| time/                   |             |
|    fps                  | 35          |
|    iterations           | 926         |
|    time_elapsed         | 53021       |
|    total_timesteps      | 1896448     |
| train/                  |             |
|    approx_kl            | 0.050804794 |
|    clip_fraction        | 0.429       |
|    clip_range           | 0.2         |
|    entropy_loss         | -7.2        |
|    explained_variance   | 0.983       |
|    learning_rate        | 0.0003      |
|    loss                 | 0.439       |
|    n_updates            | 9250        |
|    policy_gradient_loss | -0.00245    |
|    std                  | 2.97        |
|    value_loss           | 3.5         |
-----------------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 35          |
|    iterations           | 927         |
|    time_elapsed         | 53074 

----------------------------------------
| time/                   |            |
|    fps                  | 35         |
|    iterations           | 937        |
|    time_elapsed         | 53609      |
|    total_timesteps      | 1918976    |
| train/                  |            |
|    approx_kl            | 0.09473477 |
|    clip_fraction        | 0.517      |
|    clip_range           | 0.2        |
|    entropy_loss         | -7.09      |
|    explained_variance   | 0.972      |
|    learning_rate        | 0.0003     |
|    loss                 | 0.547      |
|    n_updates            | 9360       |
|    policy_gradient_loss | 0.0175     |
|    std                  | 2.85       |
|    value_loss           | 6.72       |
----------------------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 35         |
|    iterations           | 938        |
|    time_elapsed         | 53662      |
|    total_times

----------------------------------------
| time/                   |            |
|    fps                  | 35         |
|    iterations           | 948        |
|    time_elapsed         | 54200      |
|    total_timesteps      | 1941504    |
| train/                  |            |
|    approx_kl            | 0.07714959 |
|    clip_fraction        | 0.449      |
|    clip_range           | 0.2        |
|    entropy_loss         | -7.1       |
|    explained_variance   | 0.979      |
|    learning_rate        | 0.0003     |
|    loss                 | 0.553      |
|    n_updates            | 9470       |
|    policy_gradient_loss | -0.00236   |
|    std                  | 2.89       |
|    value_loss           | 4.17       |
----------------------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 35         |
|    iterations           | 949        |
|    time_elapsed         | 54254      |
|    total_times

----------------------------------------
| time/                   |            |
|    fps                  | 35         |
|    iterations           | 959        |
|    time_elapsed         | 54791      |
|    total_timesteps      | 1964032    |
| train/                  |            |
|    approx_kl            | 0.06338242 |
|    clip_fraction        | 0.46       |
|    clip_range           | 0.2        |
|    entropy_loss         | -7.13      |
|    explained_variance   | 0.97       |
|    learning_rate        | 0.0003     |
|    loss                 | 0.518      |
|    n_updates            | 9580       |
|    policy_gradient_loss | -0.000642  |
|    std                  | 2.9        |
|    value_loss           | 6.67       |
----------------------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 35         |
|    iterations           | 960        |
|    time_elapsed         | 54844      |
|    total_times

----------------------------------------
| time/                   |            |
|    fps                  | 35         |
|    iterations           | 970        |
|    time_elapsed         | 55382      |
|    total_timesteps      | 1986560    |
| train/                  |            |
|    approx_kl            | 0.03986168 |
|    clip_fraction        | 0.371      |
|    clip_range           | 0.2        |
|    entropy_loss         | -7.21      |
|    explained_variance   | 0.993      |
|    learning_rate        | 0.0003     |
|    loss                 | 0.692      |
|    n_updates            | 9690       |
|    policy_gradient_loss | -0.00339   |
|    std                  | 2.96       |
|    value_loss           | 2.68       |
----------------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 35          |
|    iterations           | 971         |
|    time_elapsed         | 55435       |
|    total_